In [1]:
import data_loader
import trainer 
from trainer import ViTBERTTrainer
import utils
import yaml
import wandb
from utils import load_config
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
from tqdm.auto import tqdm
import wandb

from data_loader import ViTBERT # Adjust the import statement according to your project structure
from model import ViTBERTClassifier # Adjust the import statement according to your project structure
from utils import get_device # Assuming you have a utility function to determine the device

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/media/data3/home/khiemdd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pathlib
import os
import torch

import torch
from torch import nn
import numpy as np
from torch.optim import Adam
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

from torchvision.transforms import v2
import torch 
import torch.optim as optim
from torchvision import transforms

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda')

In [4]:
train_dir = "/media/data3/home/khiemdd/viBert/ViTBERT/after_acronym_processed_train.csv"
test_dir = "/media/data3/home/khiemdd/viBert/ViTBERT/after_acronym_processed_test.csv"
train_data_simple =ViTBERT(data_path=train_dir,tokenizer="demdecuong/vihealthbert-base-word")
test_data_simple = ViTBERT(data_path=test_dir,tokenizer="demdecuong/vihealthbert-base-word")
# Setup batch size and number of workers 
BATCH_SIZE = 1
NUM_WORKERS = 8
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data_simple, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle= True, 
                                    num_workers=NUM_WORKERS)
print(f'train dataset size :{len(train_data_simple)}')
print(f'test dataset size :{len(test_data_simple)}')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data3/home/khiemdd/viBert/ViTBERT/after_acronym_processed_train.csv'

In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X,XN, y) in enumerate(dataloader):
        # Send data to target device
        X,XN, y = X.to(device),XN.to(device), y.to(device)
        y = y.long()
        # 1. Forward pass
        y_pred = model(X,XN)
        y_pred = y_pred.float()

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
from sklearn.metrics import f1_score

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss, test accuracy, and f1 score values
    test_loss, test_acc, total_f1_score = 0, 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, XN, y) in enumerate(dataloader):
            # Send data to target device
            X, XN, y = X.to(device), XN.to(device), y.to(device)
            y = y.long()
    
            # 1. Forward pass
            test_pred_logits = model(X, XN)
            test_pred_logits = test_pred_logits.float()

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
            
            # Calculate and accumulate F1 score
            # Note: Convert tensors to CPU numpy arrays for compatibility with sklearn
            batch_f1_score = f1_score(y.cpu().numpy(), test_pred_labels.cpu().numpy(), average='macro')
            total_f1_score += batch_f1_score
            
    # Adjust metrics to get average loss, accuracy, and F1 score per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    average_f1_score = total_f1_score / len(dataloader)
    
    # Print the average F1 score
    print(f"Average F1 Score: {average_f1_score:.4f}")
    
    return test_loss, test_acc, average_f1_score


In [ ]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    wandb.login()
    wandb.init(project='ViTBERT', config={
        "learning_rate" : 0.00001,
        "architecture": "SYLLABLE",
        "epoch": epochs,
        "batch_size": 1
    })
    results = {
        "f1_score": [],
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc, f1_score = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        wandb.log({"train loss": train_loss, "train accuracy": train_acc, "test loss" : test_loss, "test accuracy": test_acc})
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f}| "
            f"train_acc: {train_acc*100:.4f}% | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc*100:.4f}% |"
        )

        # 5. Update results dictionary
        results["f1_score"].append(f1_score)
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
def compute_and_plot_confusion_matrix(model, dataloader, device):
    # Ensure the model is in evaluation mode
    model.eval()

        
    # Initialize lists to store true and predicted labels
    true_labels = []
    pred_labels = []

    # Disable gradient computation for efficiency
    with torch.no_grad():
        for batch in dataloader:
            # Assuming each batch is a tuple of (inputs, labels)
            inputs1,inputs2, labels = batch[0].to(device), batch[1].to(device),batch[2].to(device)
            
            # Forward pass to get the model's predictions
            outputs = model(inputs1,inputs2)
            
            # Convert output probabilities to predicted class by taking the index of the max log-probability
            _, preds = torch.max(outputs, 1)
            
            # Extend the lists with the batch's true and predicted labels
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())

    # Compute the confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    
    # Plot the confusion matrix using Seaborn
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 40

# Recreate an instance of TinyVGG
model_0 =ViTBERTClassifier(num_classes=4, pretrained_model_name="demdecuong/vihealthbert-base-word").to(device)

model_0.to(device)
# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
loss_fn.cuda()
optimizer = optim.Adam(model_0.parameters(), lr=0.0000091234565434567)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: vinakhiem120. Use `wandb login --relogin` to force relogin


  5%|▌         | 1/20 [00:24<07:38, 24.11s/it]

Average F1 Score: 0.5517
Epoch: 1 | train_loss: 1.3586| train_acc: 45.8515% | test_loss: 1.3200 | test_acc: 55.1724% |


 10%|█         | 2/20 [00:47<07:01, 23.42s/it]

Average F1 Score: 0.5517
Epoch: 2 | train_loss: 1.3258| train_acc: 48.4716% | test_loss: 1.2852 | test_acc: 55.1724% |


...

Average F1 Score: 0.6207
Epoch: 18 | train_loss: 1.1293| train_acc: 68.1223% | test_loss: 1.1531 | test_acc: 62.0690% |


 95%|█████████▌| 19/20 [07:20<00:23, 23.24s/it]

Average F1 Score: 0.6379
Epoch: 19 | train_loss: 1.1374| train_acc: 65.0655% | test_loss: 1.1258 | test_acc: 63.7931% |


100%|██████████| 20/20 [07:44<00:00, 23.21s/it]

Average F1 Score: 0.6552
Epoch: 20 | train_loss: 1.1164| train_acc: 67.6856% | test_loss: 1.1162 | test_acc: 65.5172% |
Total training time: 467.341 seconds


In [ ]:
# Check the model_0_results keys
model_0_results.keys()

dict_keys(['f1_score', 'train_loss', 'train_acc', 'test_loss', 'test_acc'])

In [ ]:
import csv 
csv_filename = 'modelRESNET50_10_6_results_weightdecay.csv'
header = list(model_0_results.keys())
with open(csv_filename, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header row with the keys
    csv_writer.writerow(['Iteration'] + header)

    # Write the data for each iteration
    for i, iteration in enumerate(zip(*model_0_results.values())):
        csv_writer.writerow([i + 1] + list(iteration))

print(f"Results written to {csv_filename}")

Results written to modelRESNET50_10_6_results_weightdecay.csv


In [ ]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(model_0_results)

NameError: name 'plt' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

model_name = "microsoft/xtremedistil-l12-h384-uncased"  # Find popular HuggingFace models here: https://huggingface.co/models
input_text = "The cat sat on the mat"  
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

<IPython.core.display.Javascript object>